In [1]:
from delta import configure_spark_with_delta_pip, DeltaTable
from pyspark.sql import SparkSession

In [2]:
builder = (SparkSession.builder
           .appName("time-travel-delta-table")
           .master("spark://spark-master:7077")
           .config("spark.executor.memory", "512m")
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog"))

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3d04ed35-4571-422c-b7f0-79443f198116;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 325ms :: artifacts dl 11ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0 

In [3]:
%load_ext sparksql_magic
%config SparkSql.limit=20

In [4]:
%%sparksql
DESCRIBE DETAIL delta.`/opt/workspace/data/delta_lake/netflix_titles`

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures
delta,b9362c59-8f8a-4ced-b9a4-f4d5a4c59767,null,null,file:/opt/workspace/data/delta_lake/netflix_titles,2024-02-04 17:03:06.775000,2024-02-04 17:14:08.504000,[],1,1963749,"{'delta.deletedFileRetentionDuration': 'interval 14 days', 'delta.logRetentionDuration': 'interval 60 days'}",1,2,"['appendOnly', 'invariants']"


In [5]:
df = spark.read.format("delta").option("versionAsOf", 1).load("/opt/workspace/data/delta_lake/netflix_titles")
df.show(5)

+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                null|United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|           null|Ama Qamata, Khosi...| South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|Julien Leclercq|Sami Bouajila, Tr...|         null|Septem

In [6]:
%%sparksql
SELECT * FROM delta.`/opt/workspace/data/delta_lake/netflix_titles` VERSION AS OF 1 LIMIT 3;

show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,null,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable."
s2,TV Show,Blood & Water,null,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick Mofokeng",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth."
s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabiha Akkari, Sofia Lesaffre, Salim Kechiouche, Noureddine Farihi, Geert Van Rampelberg, Bakary Diombera",null,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Action & Adventure","To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war."


In [7]:
deltaTable = DeltaTable.forPath(spark, "/opt/workspace/data/delta_lake/netflix_titles")  # path-based tables, or
deltaTable.restoreToVersion(3) # restore table to oldest version

DataFrame[table_size_after_restore: bigint, num_of_files_after_restore: bigint, num_removed_files: bigint, num_restored_files: bigint, removed_files_size: bigint, restored_files_size: bigint]

In [8]:
%%sparksql
RESTORE TABLE delta.`/opt/workspace/data/delta_lake/netflix_titles` TO VERSION AS OF 3;

table_size_after_restore,num_of_files_after_restore,num_removed_files,num_restored_files,removed_files_size,restored_files_size
1963749,1,0,0,0,0


In [9]:
%%sparksql
DESCRIBE HISTORY "/opt/workspace/data/delta_lake/netflix_titles"

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
7,2024-02-04 17:22:01.654000,null,null,RESTORE,"{'version': '3', 'timestamp': None}",null,null,null,6,Serializable,False,"{'removedFilesSize': '0', 'tableSizeAfterRestore': '1963749', 'numRemovedFiles': '0', 'restoredFilesSize': '0', 'numOfFilesAfterRestore': '1', 'numRestoredFiles': '0'}",null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
6,2024-02-04 17:21:43.751000,null,null,RESTORE,"{'version': '3', 'timestamp': None}",null,null,null,5,Serializable,False,"{'removedFilesSize': '1963749', 'tableSizeAfterRestore': '1963749', 'numRemovedFiles': '1', 'restoredFilesSize': '1963749', 'numOfFilesAfterRestore': '1', 'numRestoredFiles': '1'}",null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
5,2024-02-04 17:14:08.504000,null,null,OPTIMIZE,"{'predicate': '[]', 'zOrderBy': '[""country""]'}",null,null,null,4,SnapshotIsolation,False,"{'numRemovedBytes': '1963749', 'p50FileSize': '1963749', 'p25FileSize': '1963749', 'numAddedFiles': '1', 'minFileSize': '1963749', 'p75FileSize': '1963749', 'numDeletionVectorsRemoved': '0', 'numAddedBytes': '1963749', 'maxFileSize': '1963749', 'numRemovedFiles': '1'}",null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
4,2024-02-04 17:14:01.672000,null,null,OPTIMIZE,"{'predicate': '[]', 'zOrderBy': '[""country""]'}",null,null,null,3,SnapshotIsolation,False,"{'numRemovedBytes': '1963749', 'p50FileSize': '1963749', 'p25FileSize': '1963749', 'numAddedFiles': '1', 'minFileSize': '1963749', 'p75FileSize': '1963749', 'numDeletionVectorsRemoved': '0', 'numAddedBytes': '1963749', 'maxFileSize': '1963749', 'numRemovedFiles': '1'}",null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
3,2024-02-04 17:05:38.590000,null,null,UPDATE,"{'predicate': '[""isnull(director#40)""]'}",null,null,null,2,Serializable,False,"{'numRemovedBytes': '1960782', 'numAddedFiles': '1', 'scanTimeMs': '2915', 'numCopiedRows': '6172', 'executionTimeMs': '5820', 'numAddedChangeFiles': '0', 'numAddedBytes': '1963749', 'numUpdatedRows': '2634', 'numRemovedFiles': '1', 'rewriteTimeMs': '2899'}",null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
2,2024-02-04 17:04:16.904000,null,null,SET TBLPROPERTIES,"{'properties': '{""delta.logRetentionDuration"":""interval 60 days"",""delta.deletedFileRetentionDuration"":""interval 14 days""}'}",null,null,null,1,Serializable,True,{},null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
1,2024-02-04 17:03:13.178000,null,null,CREATE OR REPLACE TABLE AS SELECT,"{'description': None, 'partitionBy': '[]', 'properties': '{}', 'isManaged': 'false'}",null,null,null,0,Serializable,False,"{'numOutputRows': '8806', 'numOutputBytes': '1960782', 'numFiles': '1'}",null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
0,2024-02-04 17:03:06.818000,null,null,CREATE OR REPLACE TABLE,"{'description': None, 'partitionBy': '[]', 'properties': '{}', 'isManaged': 'false'}",null,null,null,null,Serializable,True,{},null,Apache-Spark/3.4.1 Delta-Lake/2.4.0


In [10]:
spark.stop()